# Support Vector Machine

In [1]:
import pandas as pd

# Path to data file
parquet_file_path = '../../Costa_Rica_Data/Data Acquisition Output/extracted_gee_data/en_clean_gee_data.parquet'

# Load into a DataFrame
df = pd.read_parquet(parquet_file_path)

# Display head
df.head()

,plotid,sampleid,Use,CoverType,Vegetations,Herbaceous,GrasslandShrub,CropsType,WetlandArea,LandType,...,mTPI,ndvi,ocs_1mMed,sand_1mMed,savi,silt_1mMed,slope,topDiv,wetness,Forest_Presence
9,2902,11605,Wetlands,Vegetation,Trees,Not_Applicable,Not_Applicable,Not_Applicable,Swamp (Marsh),Not_Applicable,...,8129.0,0.799771,68.0,332.15,0.393728,298.85,0.927410,1323.685053,-0.006312,Present
12,2902,11608,Grasslands,Vegetation,Herbaceous plants,Grasses,Mixed Pasture (70-90%),Not_Applicable,Not_Applicable,Not_Applicable,...,8129.0,0.796553,68.0,332.15,0.423713,298.85,2.935819,1323.685053,-0.014932,Present
13,2902,11609,Wetlands,Vegetation,Herbaceous plants,Grasses,Not_Applicable,Not_Applicable,Swamp (Marsh),Not_Applicable,...,8129.0,0.644415,65.0,340.45,0.380250,304.70,0.944368,1323.685053,-0.001697,Present
15,2902,11611,Grasslands,Vegetation,Herbaceous plants,Grasses,Mixed Pasture (70-90%),Not_Applicable,Not_Applicable,Not_Applicable,...,8129.0,0.784331,68.0,332.15,0.512687,298.85,0.927410,1323.685053,-0.027987,Present
16,2902,11612,Wetlands,Vegetation,Trees,Not_Applicable,Not_Applicable,Not_Applicable,Swamp (Marsh),Not_Applicable,...,8129.0,0.769357,65.0,340.45,0.360428,304.70,2.645556,1323.685053,-0.009912,Present


In [2]:
y = df['Use']  # Set the target variable
X = df[['BLUE','GREEN', 'NIR', 'RED', 'SWIR1', 'SWIR2', 'altura2', 'aspect',
       'aspectcos', 'aspectdeg', 'brightness', 'clay_1mMed',
       'diff', 'elevation', 'evi', 'fpar', 'hand30_100', 'lai', 'mTPI', 'ndvi',
       'ocs_1mMed', 'sand_1mMed', 'savi', 'slope', 'topDiv',
       'wetness']]  # Predictor features

In [3]:
# Check for class imbalance
print("Class distribution:")
print(y.value_counts(normalize=True))

Class distribution:
Use
Forest               0.573300
Grasslands           0.237094
Agriculture          0.084957
Other classes        0.051481
Wetlands             0.039808
Forest plantation    0.009117
No information       0.002383
Not_Applicable       0.001860
Name: proportion, dtype: float64


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
# Feature scaling
from sklearn.preprocessing import StandardScaler

# standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
# Create a pipeline for SVM
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Include scaling in the pipeline
    ('svm', SVC(random_state=42))  # Replace logistic regression with SVM
])

# Cross Validation

In [7]:
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
# Splits our data into 5 folds, training on 4 parts and testing on 1
cv_scores = cross_val_score(
    svm_pipeline, X_train_scaled, y_train, cv=StratifiedKFold(n_splits=5), scoring='accuracy' # use stratified folds to keep class balance in each split
)
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean CV accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

Cross-validation scores: [0.75576256 0.75379071 0.75780241 0.75724194 0.75622195]
Mean CV accuracy: 0.7562 ± 0.0014


# Hyperparameter tuning with GridSearchCV

In [8]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

# Simplified parameter grid
param_grid = {
    'C': [1, 10],
    'kernel': ['linear']  # Start with just linear kernel
}

# Create and run RandomizedSearchCV
random_search = RandomizedSearchCV(
    SVC(random_state=42, gamma='scale'),
    param_grid,
    n_iter=2,  # Only 2 combinations to try
    cv=2,      # Reduced cross-validation folds
    n_jobs=-1,
    random_state=42
)

# Fit on a sample of data to speed things up
from sklearn.model_selection import train_test_split
X_sample, _, y_sample, _ = train_test_split(
    X_train_scaled, y_train, 
    train_size=0.1,  # Use 10% of your data
    random_state=42, 
    stratify=y_train
)

# Run the search on the sample
random_search.fit(X_sample, y_sample)

# Print results
print(f"Best parameters: {random_search.best_params_}")
print(f"Best cross-validation accuracy: {random_search.best_score_:.4f}")

Best parameters: {'kernel': 'linear', 'C': 10}
Best cross-validation accuracy: 0.7182


In [ ]:
# Train model with best parameters
best_svm = random_search.best_estimator_
best_svm.fit(X_train_scaled, y_train)

In [ ]:
# Evaluate
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
y_pred = best_svm.predict(X_test_scaled)  # Use scaled test data
accuracy = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

# PCA

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
# PCA finds the components where our data varies the most
# Helps decide how many features we actually need to keep
pca = PCA().fit(X_train_scaled)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
# Plot explained variance
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o')
plt.axhline(y=0.95, color='r', linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Explained Variance')
plt.grid(True)
plt.show()
# X-axis: number of components used
# Y-axis: percentage of total variance explained
# Red line: 95% variance threshold
# Blue line: Shows how quickly we capture information as we add more components

,ndvi,evi,savi,brightness,wetness,slope,aspect,mTPI,ocs_1mMed,sand_1mMed,topDiv
9,0.799771,0.389640,0.393728,0.213948,-0.006312,0.927410,180.000000,8129.0,68.0,332.15,1323.685053
12,0.796553,0.426831,0.423713,0.245222,-0.014932,2.935819,341.251312,8129.0,68.0,332.15,1323.685053
13,0.644415,0.414457,0.380250,0.291687,-0.001697,0.944368,90.000000,8129.0,65.0,340.45,1323.685053
15,0.784331,0.532381,0.512687,0.348042,-0.027987,0.927410,0.000000,8129.0,68.0,332.15,1323.685053
16,0.769357,0.346814,0.360428,0.198905,-0.009912,2.645556,314.480835,8129.0,65.0,340.45,1323.685053


In [ ]:
# Determine number of components for 95% variance
n_components = np.where(cumulative_variance >= 0.95)[0][0] + 1
print(f"Number of components for 95% variance: {n_components}")

Test Score: 0.7183964316797214


In [ ]:
# Create a pipeline with PCA and SVM
from sklearn.pipeline import Pipeline
# Get the best parameters from grid search
best_params = random_search.best_params_.copy()
# Remove parameters that are only used with specific kernels if necessary
if best_params.get('kernel') != 'poly' and 'degree' in best_params:
    del best_params['degree']
# Build pipeline combining PCA with SVM
pca_svm_pipeline = Pipeline([
    ('pca', PCA(n_components=n_components)),
    ('svm', SVC(**best_params, random_state=42))
])
# Train the pipeline on the scaled training data
pca_svm_pipeline.fit(X_train_scaled, y_train)
# Evaluate on test data
pca_svm_accuracy = pca_svm_pipeline.score(X_test_scaled, y_test)
print(f"PCA + SVM Test Accuracy: {pca_svm_accuracy:.4f}")

In [ ]:
print(f"Standard SVM Test Accuracy: {accuracy:.4f}")

In [ ]:
# Save the model
# import joblib
# joblib.dump(best_svm, 'best_svm_model.pkl')
# print("SVM model saved as 'best_svm_model.pkl'")